In [1]:
# install keras and seaborn in case you don't have it yet
!pip install keras

!pip install -q seaborn
!pip install pydot
!pip install tensorflow

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
dataset_path_red = keras.utils.get_file("winequality-red.csv", "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv")

dataset_path_white = keras.utils.get_file("winequality-white.csv", "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv")

column_names = ['fixed_acidity','volatile_acidity','citric_acid','residual_sugar','chlorides',
                'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
raw_dataset_red = pd.read_csv(dataset_path_red, names=column_names,
                      na_values = "None", comment='\t',
                      sep=";", skipinitialspace=False, header = 0)

dataset_red = raw_dataset_red.copy()

In [4]:
raw_dataset_white = pd.read_csv(dataset_path_white, names=column_names,
                      na_values = "None", comment='\t',
                      sep=";", skipinitialspace=False, header = 0)

dataset_white = raw_dataset_white.copy()

In [5]:
train_red = dataset_red.sample(frac=0.8,random_state=0)
test_red = dataset_red.drop(train_red.index)

train_white = dataset_white.sample(frac=0.8,random_state=0)
test_white = dataset_white.drop(train_white.index)


In [6]:
train_redq = np.array(train_red['quality'])# Remove the labels from the features
# axis 1 refers to the columns
train_red = train_red.drop('quality', axis = 1)# Saving feature names for later use
train_red_list = list(train_red.columns)# Convert to numpy array
train_red = np.array(train_red)

test_redq = np.array(test_red['quality'])# Remove the labels from the features
# axis 1 refers to the columns
test_red = test_red.drop('quality', axis = 1)# Saving feature names for later use
test_red_list = list(test_red.columns)# Convert to numpy array
test_red = np.array(test_red)

In [7]:
train_whiteq = np.array(train_white['quality'])# Remove the labels from the features
# axis 1 refers to the columns
train_white = train_white.drop('quality', axis = 1)# Saving feature names for later use
train_white_list = list(train_white.columns)# Convert to numpy array
train_white = np.array(train_white)

test_whiteq = np.array(test_white['quality'])# Remove the labels from the features
# axis 1 refers to the columns
test_white = test_white.drop('quality', axis = 1)# Saving feature names for later use
test_white_list = list(test_white.columns)# Convert to numpy array
test_white = np.array(test_white)

In [8]:
def evaluate(model, test_red, test_redq):
    predictions = model.predict(test_red)
    errors = (predictions - test_redq)**2
    mape = 100 * np.mean(errors / test_redq)
    accuracy = 100 - mape
    TSS = sum((test_redq - np.mean(test_redq))**2)
    Rsquared = 1 - sum(errors)/TSS
    print('Model Performance')
    print('Average Squared Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('R-Squared = {:0.2f}.'.format(Rsquared))
    return accuracy

In [ ]:
def randomforest(train_redq, train_red, test_redq, test_red):
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    
    bootstrap = [True, False]# Create the random grid
    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
    
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
    rf_random.fit(train_red, train_redq)
    best_random = rf_random.best_params_
    random_accuracy = evaluate(best_random, test_red, test_redq)
    return random_accuracy

In [ ]:
randomforest(train_redq, train_red, test_redq, train_red)

In [ ]:
randomforest(train_whiteq, train_white, test_redq, train_red)